# PREPROCESSING

- [Importar librerías](#Importar-librerías)
- [Lectura de los datasets](#Lectura-de-los-datasets)
- [Eliminación de características irrelevantes](#Eliminación-de-características-irrelevantes)
- [Manejo de datos faltantes](#Manejo-de-datos-faltantes)
- [Manejo de outliers](#Manejo-de-outliers)
- [Gestión de tipos](Gestión-de-tipos)
- [Codificación-de-variables-categóricas](Codificación-de-variables-categóricas)
- [Normalización y estandarización](#Normalización-y-estandarización)
- [Transformaciones de datos](#Transformaciones-de-datos)

## Importar librerías

!pip install -r requirements.txt

In [1]:
import os
import shutil
import zipfile
import pandas as pd

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

## Lectura de los datasets

In [2]:
INPUT_ZIP = "./00_Data/Raw/titanic.zip"  # Directorio del zip
OUTPUT_FOLDER = "./00_Data/Raw"  # Directorio de destino
TRAIN_FILENAME = "train.csv"  # Nombre del fichero de entrenamiento
TEST_FILENAME = "test.csv"  # Nombre del fichero de entrenamiento

def fetch_data(input_path=INPUT_ZIP, output_dir=OUTPUT_FOLDER):
    """
    Extrae el contenido de un archivo ZIP en un directorio de destino.

    Parámetros:
    -----------
    input_path : str, opcional
        Ruta al archivo ZIP que se desea descomprimir. El valor predeterminado es la variable 'INPUT_ZIP'.
        
    output_dir : str, opcional
        Directorio en el cual se extraerá el contenido del archivo ZIP. Si el directorio no existe,
        será creado automáticamente. El valor predeterminado es la variable 'OUTPUT_FOLDER'.

    Comportamiento:
    ---------------
    - Crea el directorio de destino si no existe.
    - Descomprime el archivo ZIP en el directorio de destino.

    Excepciones:
    ------------
    Puede lanzar una excepción si el archivo ZIP no existe o si hay problemas al descomprimirlo.

    Ejemplo de uso:
    ---------------
    fetch_data('data.zip', 'output/')
    """
    # Comprobación de que el directorio de destino existe
    os.makedirs(output_dir, exist_ok=True)

    # Descomprime el archivo ZIP en caso de que no haya ningún csv en la carpeta
    if(len([file for file in os.listdir(output_dir) if file.endswith('.csv')]) == 0):
        with zipfile.ZipFile(input_path, 'r') as zip_ref:
            zip_ref.extractall(output_dir)


def load_data(directory=OUTPUT_FOLDER, filename=TRAIN_FILENAME):
    """
    Lee un archivo CSV desde el directorio especificado.

    Parámetros:
    -----------
    directory : str
        El directorio donde se encuentra el archivo CSV.
        
    filename : str
        El nombre del archivo CSV a leer (incluyendo la extensión .csv).

    Retorna:
    --------
    pd.DataFrame
        Un DataFrame de pandas que contiene los datos del archivo CSV.

    Excepciones:
    ------------
    FileNotFoundError:
        Se lanza si el archivo no existe en el directorio dado.
    
    Ejemplo de uso:
    ---------------
    df = read_csv_from_directory('data', 'file.csv')
    """
    # Construir la ruta completa al archivo CSV
    file_path = os.path.join(directory, filename)

    # Verificar si el archivo existe
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"El archivo {filename} no se encuentra en el directorio {directory}")

    # Leer el archivo CSV en un DataFrame
    return pd.read_csv(file_path)

fetch_data()
df_train = load_data(OUTPUT_FOLDER, TRAIN_FILENAME)
df_test = load_data(OUTPUT_FOLDER, TEST_FILENAME)

print("Train dataset:", df_train.shape)
print("Test dataset:", df_test.shape)

Train dataset: (891, 12)
Test dataset: (418, 11)


Como vemos, en el dataset de test no se incluyen las 12 columnas (quitan la de predicción)

## Eliminación de características irrelevantes

In [3]:
# Seleccionamos las columnas con las que nos queremos quedar
cols_keep = ["Sex", "Pclass", "Age", "SibSp", "Parch", "Embarked", "Cabin", "Fare"]
predict_col = "Survived"

In [4]:
print("Las columnas originales son:", df_train.columns.tolist())

df_train = df_train[cols_keep + [predict_col]]
df_test = df_test[cols_keep]

print("Las columnas tras eliminar irrelevantes son:", df_train.columns.tolist())

Las columnas originales son: ['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
Las columnas tras eliminar irrelevantes son: ['Sex', 'Pclass', 'Age', 'SibSp', 'Parch', 'Embarked', 'Cabin', 'Fare', 'Survived']


## Manejo de datos faltantes

In [5]:
df_train.isnull().sum()

Sex           0
Pclass        0
Age         177
SibSp         0
Parch         0
Embarked      2
Cabin       687
Fare          0
Survived      0
dtype: int64

In [6]:
df_test.isnull().sum()

Sex           0
Pclass        0
Age          86
SibSp         0
Parch         0
Embarked      0
Cabin       327
Fare          1
dtype: int64

In [7]:
# Definimos función para imputación simple
def simple_imputation(col_name, training, test, strategy='median'):
    """
    Realiza una imputación simple de valores faltantes en una columna específica de los 
    conjuntos de entrenamiento y test, utilizando la estrategia especificada.

    Args:
        strategy (str, opcional): La estrategia de imputación a utilizar. Puede ser 'mean', 
                                  'median', 'most_frequent', o 'constant'. Por defecto es 'median'.
        col_name (str): El nombre de la columna sobre la cual se realizará la imputación.
        training (pd.DataFrame): El conjunto de datos de entrenamiento que contiene la columna con valores faltantes.
        test (pd.DataFrame): El conjunto de datos de prueba que contiene la columna con valores faltantes.

    Returns:
        tuple: Una tupla que contiene los conjuntos de datos de entrenamiento y prueba después de aplicar la imputación.

    Raises:
        ValueError: Si el nombre de la columna no existe en los DataFrames proporcionados.
    """

    # Definición del imputador
    imputer = SimpleImputer(strategy=strategy)

    # Entrenamiento del imputer + transformación en training
    training[col_name] = imputer.fit_transform(training[[col_name]]).ravel()  # ravel para aplanar la matriz 2D

    # Transformación en test
    test[col_name] = imputer.transform(test[[col_name]]).ravel()  # ravel para aplanar la matriz 2D

    return training, test

In [8]:
# Imputamos en columna 'Age' con la mediana
df_train, df_test = simple_imputation('Age', df_train, df_test)

In [9]:
# Con cabin, puesto que creamos variable auxiliar de 'Has_Cabin', lo hacemos aquí y eliminamos la original
df_train['Has_Cabin'] = df_train['Cabin'].notnull().astype(int)
df_train = df_train.drop('Cabin', axis=1)
df_test['Has_Cabin'] = df_test['Cabin'].notnull().astype(int)
df_test = df_test.drop('Cabin', axis=1)

In [10]:
# Imputamos en columna 'Embarked' con la moda (categórica)
df_train, df_test = simple_imputation('Embarked', df_train, df_test, 'most_frequent')

In [11]:
# Al detectar que tenemos un missing value en test para 'Fare', lo imputamos con la mediana de training
_, df_test = simple_imputation('Fare', df_train, df_test)

In [12]:
# Check para ver si hemos imputado correctamente en train
df_train.isnull().sum()

Sex          0
Pclass       0
Age          0
SibSp        0
Parch        0
Embarked     0
Fare         0
Survived     0
Has_Cabin    0
dtype: int64

In [13]:
# Check para ver si hemos imputado correctamente en test
df_test.isnull().sum()

Sex          0
Pclass       0
Age          0
SibSp        0
Parch        0
Embarked     0
Fare         0
Has_Cabin    0
dtype: int64

## Manejo de outliers

En este caso hemos estudiado los outliers y parece que son desviaciones estadísticas, nada que necesitemos eliminar o imputar para nuestro problema.

## Gestión de tipos

El único tipo que teníamos que cambiar era el de Age a integer una vez imputásemos los missing values, pero vamos a crear la columna 'Age_bin' y eliminar esta, por lo que no necesitamos hacerlo.

## Codificación de variables categóricas

In [14]:
def one_hot_encode_column(col_name, training, test):
    """
    Aplica One-Hot Encoding a una columna categórica en los DataFrames de entrenamiento y prueba.
    
    La función codifica la columna indicada en ambos DataFrames utilizando One-Hot Encoding
    con el encoder de scikit-learn. Asegura que ambos DataFrames tengan las mismas columnas 
    después de la codificación. Además, elimina una columna codificada para evitar multicolinealidad,
    y convierte los valores resultantes a enteros.

    Args:
        col_name (str): El nombre de la columna categórica a codificar.
        training (pd.DataFrame): El DataFrame de entrenamiento.
        test (pd.DataFrame): El DataFrame de prueba.

    Returns:
        tuple: Una tupla que contiene dos DataFrames (entrenamiento y prueba) 
               con la columna especificada codificada mediante One-Hot Encoding.
    """
    
    # Inicializar OneHotEncoder
    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')  # Sparse=False para obtener un array de numpy; handle_unknown='ignore' para ignorar valores desconocidos
    
    # Ajustar el encoder con el DataFrame de entrenamiento y transformar
    training_encoded = encoder.fit_transform(training[[col_name]])
    test_encoded = encoder.transform(test[[col_name]])  # Usar transform para test sin ajustar de nuevo
    
    # Convertir los resultados codificados a DataFrames con los nombres de columnas adecuados
    training_encoded_df = pd.DataFrame(training_encoded, 
                                       columns=encoder.get_feature_names_out([col_name]),
                                       index=training.index)
    
    test_encoded_df = pd.DataFrame(test_encoded, 
                                   columns=encoder.get_feature_names_out([col_name]),
                                   index=test.index)
    
    # Comprobar si todas las columnas son nulas en test (categoría desconocida)
    if test_encoded_df.isnull().all().any():
        print('Variable desconocida en test')
        test_encoded_df[col_name + '_desconocidos'] = (test_encoded_df.sum(axis=1) == 0).astype(int)
        training_encoded_df[col_name + '_desconocidos'] = 0       

    # Convertir los valores codificados a enteros
    training_encoded_df = training_encoded_df.astype(int)
    test_encoded_df = test_encoded_df.astype(int)
    
    # Eliminar una columna codificada para evitar multicolinealidad (por ejemplo, la primera columna)
    training_encoded_df = training_encoded_df.drop(columns=encoder.get_feature_names_out([col_name])[0])
    test_encoded_df = test_encoded_df.drop(columns=encoder.get_feature_names_out([col_name])[0])
    
    # Combinar los DataFrames codificados con los DataFrames originales, excluyendo la columna original
    training_combined = pd.concat([training.drop(columns=[col_name]), training_encoded_df], axis=1)
    test_combined = pd.concat([test.drop(columns=[col_name]), test_encoded_df], axis=1)
    
    # Retornar los DataFrames de entrenamiento y prueba codificados
    return training_combined, test_combined


In [15]:
# Codificación de 'Sex' (one-hot)
df_train, df_test = one_hot_encode_column('Sex', df_train, df_test)

In [16]:
# Codificación de 'Embarked' (one-hot)
df_train, df_test = one_hot_encode_column('Embarked', df_train, df_test)

## Normalización y estandarización

Hemos transformado todas las variables numéricas en categorías, por lo que no hay que normalizar/estandarizar

## Transformaciones de datos

In [17]:
# Transformación de Age -> Age_bin (cambio ['Child', 'Teenager', 'Adult', 'Elderly'] -> [0, 1, 2, 3] para tenerlo codificado)
df_train['Age_bin'] = pd.cut(df_train['Age'], bins=[0, 12, 18, 60, 80], labels=[0, 1, 2, 3])
df_train = df_train.drop('Age', axis=1)
df_test['Age_bin'] = pd.cut(df_test['Age'], bins=[0, 12, 18, 60, 80], labels=[0, 1, 2, 3])
df_test = df_test.drop('Age', axis=1)

In [18]:
# Creación de 'FamilySize'
df_train['FamilySize'] = df_train['SibSp'] + df_train['Parch'] + 1
df_train = df_train.drop(columns=['SibSp', 'Parch'], axis=1)
df_test['FamilySize'] = df_test['SibSp'] + df_test['Parch'] + 1
df_test = df_test.drop(columns=['SibSp', 'Parch'], axis=1)

In [19]:
# Creación de 'Fare_Range' y codificación

# Calcular los cuartiles en df_train y recuperar los límites
_, bins = pd.qcut(df_train['Fare'], 4, retbins=True)  # retbins=True para obtener los límites de los cuartiles generados
df_train['Fare_Range'] = pd.cut(df_train['Fare'], bins=bins)
df_train = df_train.drop('Fare', axis=1)

# Aplicar los mismos límites a df_test usando pd.cut
df_test['Fare_Range'] = pd.cut(df_test['Fare'], bins=bins)
df_test = df_test.drop('Fare', axis=1)

# Codificación de la nueva variable
fare_encoder = LabelEncoder()
df_train['Fare_Range'] = fare_encoder.fit_transform(df_train['Fare_Range'])
df_test['Fare_Range'] = fare_encoder.transform(df_test['Fare_Range'])

In [20]:
df_train

,Pclass,Survived,Has_Cabin,Sex_male,Embarked_Q,Embarked_S,Age_bin,FamilySize,Fare_Range
0,3,0,0,1,0,1,2,2,0
1,1,1,1,0,0,0,2,2,3
2,3,1,0,0,0,1,2,1,1
3,1,1,1,0,0,1,2,2,3
4,3,0,0,1,0,1,2,1,1
...,...,...,...,...,...,...,...,...,...
886,2,0,0,1,0,1,2,1,1
887,1,1,1,0,0,1,2,1,2
888,3,0,0,0,0,1,2,4,2
889,1,1,1,1,0,0,2,1,2


In [21]:
df_test

,Pclass,Has_Cabin,Sex_male,Embarked_Q,Embarked_S,Age_bin,FamilySize,Fare_Range
0,3,0,1,1,0,2,1,0
1,3,0,0,0,1,2,2,0
2,2,0,1,1,0,3,1,1
3,3,0,1,0,1,2,1,1
4,3,0,0,0,1,2,3,1
...,...,...,...,...,...,...,...,...
413,3,0,1,0,1,2,1,1
414,1,1,0,0,0,2,1,3
415,3,0,1,0,1,2,1,0
416,3,0,1,0,1,2,1,1


## Escritura de los dataframes resultantes

In [22]:
OUTPUT_FOLDER = "./00_Data/Cleaned/"

def save_dataframes_to_csv(output_folder, df_train, df_test, train_filename="train_clean.csv", test_filename="test_clean.csv"):
    """
    Guarda los DataFrames de entrenamiento y prueba en formato CSV en una carpeta específica.
    Si la carpeta ya existe, borra todo su contenido antes de guardar los nuevos archivos.
    
    Args:
        output_folder (str): La ruta de la carpeta donde se guardarán los archivos CSV.
        df_train (pd.DataFrame): El DataFrame de entrenamiento que se va a guardar.
        df_test (pd.DataFrame): El DataFrame de prueba que se va a guardar.
        train_filename (str, opcional): El nombre del archivo CSV para el DataFrame de entrenamiento.
        test_filename (str, opcional): El nombre del archivo CSV para el DataFrame de prueba.
    
    """
    # Si la carpeta ya existe, eliminar todo su contenido
    if os.path.exists(output_folder):
        shutil.rmtree(output_folder)  # Borrar toda la carpeta y su contenido
        print(f"Carpeta {output_folder} eliminada.")
    
    # Crear la carpeta si no existe
    os.makedirs(output_folder, exist_ok=True)
    
    # Definir las rutas completas de los archivos
    train_path = os.path.join(output_folder, train_filename)
    test_path = os.path.join(output_folder, test_filename)
    
    # Guardar los DataFrames en formato CSV
    df_train.to_csv(train_path, index=False)
    df_test.to_csv(test_path, index=False)
    
    print(f"DataFrames guardados en {output_folder}:")
    print(f" - {train_filename}")
    print(f" - {test_filename}")

save_dataframes_to_csv(OUTPUT_FOLDER, df_train, df_test)

Carpeta ./00_Data/Cleaned/ eliminada.
DataFrames guardados en ./00_Data/Cleaned/:
 - train_clean.csv
 - test_clean.csv
